# Recuperación de pacientes del archivo de XiO
---------

# Procedimiento

- Configurar las herramientas necesarias
- Recuperar mediante FTP los datos archivados de XiO (herramientas externas)
- Subir a **gDrive** de Radiofísica los datos archivados
- Introducir los parámetros relevantes
- Correr los scripts de recuparación
- Descargar los archivos recuperados
- Limpiar en gDrive los archivos creados

## Configuración de herramientas

Instalación de `plastimatch` y `tree`

In [5]:
%%capture
#!sudo apt update

!sudo apt install plastimatch

!sudo apt install tree

Instalación e importación de los módulos `pydicom` y `pyplastimatch`

In [6]:
%%capture

# Instalación de módulos
%pip install pydicom
%pip install pyplastimatch

# Importación de módulos
# - Manipulación de archivos DICOM
import pydicom as dicom
# - Conversión del formato nativo de XiO a DICOM
import pyplastimatch as plast

Cargar los módulos necesrios

In [7]:
# - Acceso a los datos en gDrive
from google.colab import drive
# - Rutas y comprobaciones de existencia de ficheros
from os import path
from pathlib import Path
from glob import glob
from sys import path as syspath
# - Utilidades de archivo entre gDrive y el sistema local de archivos
from google.colab import files

Montar el disco

In [8]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Crear uan referencia al directorio base de trabajo, crearlo e ir a él

In [9]:
%%capture
# Referenciar
baseworkdir = '/content/gdrive/MyDrive/XiO'

# Crear el directorio
!mkdir -p $baseworkdir

# Ir al directorio base
%cd $baseworkdir

Crear la carpeta `scripts` en el directorio base de trabajo en **gDrive** y descargar los scripts que complementan el funcionamiento de `plastimatch`

In [10]:
%%capture
# Descargar los scripts de corrección
if Path(baseworkdir + '/scripts').exists():
  !rm -rf $baseworkdir/scripts
!mkdir -p $baseworkdir/scripts
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/xioss
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/genindex
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/deltact
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/dcmcoher.py


# Añadir al sistema la ruta de los scripts
if baseworkdir + '/scripts' not in syspath:
  syspath.insert(0, baseworkdir + '/scripts')

# Comprobar si el plan genérico con un único campo está y si no descargarlo
if not Path('RTPlan.dcm').exists():
  !wget https://raw.githubusercontent.com/csarux/XiOrecovery/main/DICOMSets/RTPlan.dcm

## Subir a **gDrive** de Radiofísica los datos archivados

Arrastrar sobre la carpeta XiO el archivo `.tgz` recuperado

Listar pacientes archivados presentes en el directorio base

In [11]:
!ls -l $baseworkdir/*.tgz

-rw------- 1 root root 1398081571 Jun 16 11:31 /content/gdrive/MyDrive/XiO/185834_ALONSO_ANTON_MILAGROS_Oct_17_2011.tgz


Definir el fichero que contiene el nombre del paciente archivado

## Introducir los parámetros relevantes

In [12]:
ArchivedPatient = '185834_ALONSO_ANTON_MILAGROS_Oct_17_2011.tgz'  #@param {type: "string"}

Extaer los datos archivados

In [13]:
%%capture
!tar -xvzf $ArchivedPatient

Inferir el nýumero de historia a partir del fichero archivado

In [14]:
PatientID = ArchivedPatient.split('_')[0]
PatientID

'185834'

Recuperar los apellidos y nombre del paciente almacenados en el fichero `demographic` de XiO

In [15]:
demographicfile = !find ./patient/$PatientID -name demographic -print
with open(demographicfile[0]) as demogf:
  demographicdata = demogf.read()

ApellidosNombre = demographicdata.split('\n')[2]
ApellidosNombre

'ALONSO ANTON,MILAGROS'

Visualizar la estructura de carpetas creadas

In [16]:
!tree -d patient/185834

patient/185834
├── anatomy
│   └── studyset
│       ├── CT1
│       │   └── surfaces
│       ├── CT2
│       │   ├── DCMData
│       │   └── surfaces
│       └── CT3
│           ├── DCMData
│           └── surfaces
├── dvh
└── plan
    ├── IMRT2CBCT
    ├── IMRT3
    ├── IMRT4
    ├── IMRT5
    ├── IMRTCBCT
    ├── PLANSETUPIMRT4
    ├── PLANSETUPIMRTC
    ├── PTV29CON4GY
    ├── SETUP29
    ├── SUMA
    ├── SUMA2
    ├── sumaimrts1y4
    ├── TMP12TTO
    ├── TMP3
    ├── TMP4
    └── TMP5

28 directories


Definir las variables con la identificación del estudio CT y el nombre del plan

In [23]:
CT = 'CT1' #@param {type: "string"}
Plan ='IMRTCBCT' #@param {type: "string"}

Definir las variables de la prescripción

In [18]:
DosisTotal = 66.  #@param {type: "number"}
NumeroSesiones = 33 #@param {type: "integer"}
DosisSesion =2.0 #@param {type: "number"}

## Correr los scripts de recuperación

**Estrategia**

- Crear el índice de las imágenes de CT si no existe
- Recuperar las estructuras a partir de los archivos de texto `.msh`
- Transformar estructuras y dosis a formato DICOM
- Introducir un plan básico genérico y asociarle a la dosis recpuerada y al conjunto de estructuras
- Establecer una marco de referencia coherente entre las imágenes CT y el conjunto plan, dosis, estructuras

####Reacondicionamiento de los archivos de las estructuras

In [20]:
# Si no existe crear el índice de imágenes CT y las máscaras `.mha` de todas las estructuras
if not Path(baseworkdir + '/patient/' + PatientID + '/anatomy/studyset/' + CT + '/index.dat').exists():
  # Copiar el script para la creación del índice de cortes de CT
  !cp scripts/genindex patient/$PatientID/anatomy/studyset/$CT

  # Cambiar el directorio de trabajo
  %cd patient/$PatientID/anatomy/studyset/$CT

  # Adjudicar permisos de ejecución y crear el índice
  !sudo chmod +x genindex
  !./genindex

# Cambiar el directorio de trabajo
%cd patient/$PatientID/anatomy/studyset/$CT

# Crear la carpeta para los archivos máscara con las estructuras
!mkdir -p ./ss

# Copiar el script para la creación de las estructuras
!cp $baseworkdir/scripts/xioss .

# Correr el script de recuperación de las estructuras
!sudo chmod +x xioss
!./xioss --input $baseworkdir/patient/$PatientID --plan $Plan --studyset $CT > xioss.log

/content/gdrive/MyDrive/XiO/patient/185834/anatomy/studyset/CT1
mv: 'ss/SPINAL CORD.mha' and 'ss/SPINAL CORD.mha' are the same file
mv: 'ss/externo.mha' and 'ss/externo.mha' are the same file
mv: 'ss/patient.mha' and 'ss/patient.mha' are the same file
mv: 'ss/MANDIBLE.mha' and 'ss/MANDIBLE.mha' are the same file
mv: 'ss/CTV ganglios.mha' and 'ss/CTV ganglios.mha' are the same file
mv: 'ss/PTV 54.mha' and 'ss/PTV 54.mha' are the same file
mv: 'ss/ptv54plan.mha' and 'ss/ptv54plan.mha' are the same file
mv: 'ss/mandibulaplan.mha' and 'ss/mandibulaplan.mha' are the same file
mv: 'ss/LEFT ORBIT.mha' and 'ss/LEFT ORBIT.mha' are the same file
mv: 'ss/BRAINSTEM.mha' and 'ss/BRAINSTEM.mha' are the same file
mv: 'ss/LEFT PAROTID.mha' and 'ss/LEFT PAROTID.mha' are the same file
mv: 'ss/RIGHT PAROTID.mha' and 'ss/RIGHT PAROTID.mha' are the same file
mv: 'ss/target vol. 1.mha' and 'ss/target vol. 1.mha' are the same file
mv: 'ss/PTV 66.mha' and 'ss/PTV 66.mha' are the same file
mv: 'ss/TIROIDES.mha

####Recuperación del CT, las estructuras y la dosis

In [24]:
# Regresar al directorio base
%cd $baseworkdir

# Variables de pyplastimatch
verbose = True
path_to_logfile_xiodcm = './' + PatientID + '_xiodcm.log'

# Opciones de plastimatch convert
convert_args_rt ={'input' :  'patient/' + PatientID  + '/anatomy/studyset/' + CT + '/DCMData',
                  'input-prefix' : 'patient/' + PatientID  + '/anatomy/studyset/' + CT + '/ss',
                  'input-dose-xio' : 'patient/' + PatientID  + '/plan/' + Plan + '/dose.1',
                  'output-dicom' : 'xiodcm'}

# Correr el script de conversión de los datos XiO en estudio DICOM
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )


/content/gdrive/MyDrive/XiO

Running 'plastimatch convert' with the specified arguments:
  --input patient/185834/anatomy/studyset/CT1/DCMData
  --input-prefix patient/185834/anatomy/studyset/CT1/ss
  --input-dose-xio patient/185834/plan/IMRTCBCT/dose.1
  --output-dicom xiodcm
... Done.


####Añadir un plan genérico y fijar un marco coherente de referencia

In [25]:
%cd patient

/content/gdrive/MyDrive/XiO/patient


In [26]:
# Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
!cp $baseworkdir/scripts/deltact .

## Asegurar que el archivo se actualiza
if Path('./deltaframes').exists():
  !rm -f ./deltaframes
## Adjudicar permisos de ejecución y calcular el delta
!sudo chmod +x deltact
!./deltact --input $PatientID --studyset $CT > deltact.log

## Regresar al directorio base
%cd $baseworkdir

# Incorporar el plan genérico
!cp RTPlan.dcm xiodcm

# Importar las herramientas de corrección
from dcmcoher import correctImagePositionPatientInCTImages, correctRTPlan

# Asociar al plan las dosis y estructuras
correctRTPlan()

# Establecer el mismo marco de referencia en todo el estudio
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT)

/content/gdrive/MyDrive/XiO


## Descargar los archivos recuperados

In [27]:
!zip -rq xiodcm.zip xiodcm
files.download('xiodcm.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Limpiar gDrive

In [ ]:
!rm -rf xiodcm patient xiodcm.zip *.log $ArchivedPatient

In [ ]:
!rm -rf xiodcm patient xiodcm.zip *.log

In [28]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

In [31]:
file_list = gdrive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  if file1['title'] == 'XiO' and file1['mimeType']=='application/vnd.google-apps.folder': file1.Delete()



-------
## Logfiles

Revisión de *logfiles*

Log del plan

In [29]:
!cat $PatientID'_xiodcm.log'

Could not open input file patient/185834/anatomy/studyset/CT1/DCMData for read
Loading structure: patient
Loading structure: target vol. 1
Origin = -250.0000 -250.0000 -270.0000
Size = 512 512 121
Spacing = 0.9766 0.9766 3.0000
Direction = 1.0000 0.0000 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000 1.0000
Loading structure: LEFT ORBIT
Origin = -250.0000 -250.0000 -270.0000
Size = 512 512 121
Spacing = 0.9766 0.9766 3.0000
Direction = 1.0000 0.0000 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000 1.0000
Loading structure: BRAINSTEM
Origin = -250.0000 -250.0000 -270.0000
Size = 512 512 121
Spacing = 0.9766 0.9766 3.0000
Direction = 1.0000 0.0000 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000 1.0000
Loading structure: LEFT COCHLEA
Origin = -250.0000 -250.0000 -270.0000
Size = 512 512 121
Spacing = 0.9766 0.9766 3.0000
Direction = 1.0000 0.0000 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000 1.0000
Loading structure: RIGHT COCHLEA
Origin = -250.0000 -250.0000 -270.0000
Size = 512 512 121
Spacing = 0.9766 0.9766

Log de estructuras

In [30]:
!cat patient/$PatientID/anatomy/studyset/$CT/xioss.log

Structures in contournames file:
1 => patient
2 => tumor
3 => target vol. 1
4 => target vol. 2
5 => target vol. 3
6 => target vol. 4
7 => target vol. 5
8 => BRAIN
9 => LEFT TEMPORAL LOBE
10 => RIGHT TEMPORAL LOBE
11 => LEFT ORBIT
12 => RIGHT ORBIT
13 => LEFT LENS
14 => RIGHT LENS
15 => LEFT OPTIC NERVE
16 => RIGHT OPTIC NERVE
17 => BRAINSTEM
18 => OPTIC CHIASM
19 => LEFT COCHLEA
20 => RIGHT COCHLEA
21 => MANDIBLE
22 => RIGHT PAROTID
23 => LEFT PAROTID
24 => LEFT RETROPHARYNGEAL LEVEL
25 => RIGHT RETROPHARYNGEAL LEVEL
26 => SUPERIOR CONSTRICTORS
27 => SPINAL CORD
28 => RIGHT LEVEL IIB
29 => SPINAL CAVITY
30 => LEFT LEVEL IIB
31 => LEFT LEVEL IIA
32 => RIGHT LEVEL IIA
33 => RIGHT SUBMANDIBULAR GLAND
34 => LEFT SUBMANDIBULAR GLAND
35 => RIGHT LEVEL IB
36 => LEFT LEVEL IB
37 => BRACHIAL PLEXUS
38 => MIDDLE CONSTRICTORS
39 => RIGHT LEVEL III
40 => LEFT LEVEL III
41 => RIGHT LEVEL V
42 => LEFT LEVEL V
43 => INFERIOR CONSTRICTORS
44 => LEVEL IA
45 => LARYNX
46 => LEVEL VI
47 => LEFT LEVEL IV
